# Pseudonymous Access To Encrypted Table

This notebook shows how to use [pgsodium]() and [postgresql-anonymizer]() to provide pseudonymous access to encrypted table data. `pgsodium` is used to encrypt the data, and the `anon` extension is used to anonymize it, and to make some fake test data as you'll see.  First we'll load the sql extension for jupyter and connect to the database running with this container.  The source code can be found [here]().

In [1]:
%load_ext sql
%sql postgresql://postgres@/
%sql -f pgsodium_anonymizer.sql

 * postgresql://postgres@/
Done.
Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Demo Data

Now make some demo data with the provided function.  This will create N number of encrypted demo records.

In [2]:
%sql select demo_data(10)
%sql select count(*) from encrypted_record

 * postgresql://postgres@/
1 rows affected.
 * postgresql://postgres@/
1 rows affected.


count
10


## Access Decrypted Data

While still the `postgres` superuser role, we can see the raw decrypted data.  This is the data we want to protect from other roles in the system like `staff`.

In [3]:
%sql select rec.* from encrypted_record e left join lateral decrypt_record(e.id) rec on true limit 10;

 * postgresql://postgres@/
10 rows affected.


id,first_name,last_name,age,secret,signup_date
1,yahya,Chiotti,95,Kālābāgh,2018-03-02 17:07:05.124475+00:00
2,daley,Chiotti,53,Mount Lebanon,2013-03-08 13:57:39.824305+00:00
3,pierina,Horenstein,14,Narangba,2014-10-25 03:53:20.076964+00:00
4,jouko,Defiore,94,Kalocsa,2013-03-07 04:15:50.335154+00:00
5,ilana,Bagley,26,Mitras Poniente,2015-07-06 16:24:43.602278+00:00
6,anatoli,Wesolowski,70,Talca,2016-02-22 05:12:03.492212+00:00
7,zayd,Lemarr,87,Almansa,2014-05-28 13:57:14.294830+00:00
8,sanna,Touchette,74,Novokhovrino,2013-03-03 09:12:07.639588+00:00
9,aaliyah,Feldmann,40,Monte Azul Paulista,2014-09-21 08:15:34.225063+00:00
10,della,Calderone,54,Teshi Old Town,2012-09-09 21:35:57.563589+00:00


## Restrict Access by Role

After switching roles with `SET ROLE` the `staff` role can only call the `pseudo_record()` function, it cannot access the `encrypted_record` table or the `decrypt_record` function at all.

In [4]:
%sql SET ROLE staff
%sql select * from encrypted_record
%sql select decrypt_record(1)

 * postgresql://postgres@/
Done.
 * postgresql://postgres@/
(psycopg2.errors.InsufficientPrivilege) permission denied for table encrypted_record

[SQL: select * from encrypted_record]
(Background on this error at: http://sqlalche.me/e/13/f405)
 * postgresql://postgres@/
(psycopg2.errors.InsufficientPrivilege) permission denied for function decrypt_record

[SQL: select decrypt_record(1)]
(Background on this error at: http://sqlalche.me/e/13/f405)


## Pseudonymous Access

Now as the `staff` role, you only access pseudonymous data. Notice how the names are different, the age is a range, the secret is obscured, and the signup date is shown as a month.

In [5]:
%sql select rec.* from encrypted_record e left join lateral pseudo_record(e.id) rec on true limit 10;

 * postgresql://postgres@/
10 rows affected.


id,first_name,last_name,age_range,secret,signup_month
1,gioachino,Lauderman,"[95, 100)",Kāxxxxxxxxxgh,2018-03-01 00:00:00+00:00
2,kanti,Lauderman,"[50, 55)",Moxxxxxxxxxon,2013-03-01 00:00:00+00:00
3,pleasance,Warehime,"[10, 15)",Naxxxxxxxxxba,2014-10-01 00:00:00+00:00
4,dalal,Pettway,"[90, 95)",Kaxxxxxxxxxsa,2013-03-01 00:00:00+00:00
5,serafima,Fitzen,"[25, 30)",Mixxxxxxxxxte,2015-07-01 00:00:00+00:00
6,cináed,Lydon,"[70, 75)",Taxxxxxxxxxca,2016-02-01 00:00:00+00:00
7,grania,Mcmanigal,"[85, 90)",Alxxxxxxxxxsa,2014-05-01 00:00:00+00:00
8,ingrid,Nastasia,"[70, 75)",Noxxxxxxxxxno,2013-03-01 00:00:00+00:00
9,laurelle,Malsom,"[40, 45)",Moxxxxxxxxxta,2014-09-01 00:00:00+00:00
10,wilton,Hoscheit,"[50, 55)",Texxxxxxxxxwn,2012-09-01 00:00:00+00:00
